# tc3-as05
Atividade de Tópicos em Computação III - AS05: Implementação de Assistente Conversacional Baseado em LLM.
Código feito no google colab.

## Execução
Para execução do código é necessário:
- Adição das variáveis de ambiente 'PINECONE_API_KEY' e 'GOOGLE_API_KEY';
- Executar o código;
- Colocar entradas no input:
  - 'sair' para fechar o programa;
  - 'anexar' para adicionar arquivos ao repositório;
  - A entrada desejada para análise dos arquivos anexados.

#Functions

In [1]:
%pip install -qU langchain-pinecone pinecone-notebooks

from pinecone import Pinecone, ServerlessSpec
import time
from langchain_pinecone import PineconeVectorStore
import os

def set_up_pinecone(index_name):
  pc = Pinecone(api_key=os.environ["PINECONE_API_KEY"])
  existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]
  if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=768,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)
  return pc.Index(index_name)

def get_vector_store(index, embeddings):
  vector_store = PineconeVectorStore(index=index, embedding=embeddings)
  return vector_store

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 4.4 MB/s eta 0:00:00


In [2]:
!pip install -qU langchain-google-genai

import google.generativeai as genai
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
import os

def set_up_gemini():
  genai.configure(api_key=os.environ["GOOGLE_API_KEY"])


def get_gemini_embeddings():
  genai.configure(api_key=os.environ["GOOGLE_API_KEY"])
  return GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")

def create_chat_model():
  return ChatGoogleGenerativeAI(
      model="gemini-1.5-flash",
      temperature=0,
      max_tokens=None,
      timeout=None,
      max_retries=2,
      # other params...
  )

def create_chat_prompt(template, query):
  return ChatPromptTemplate.from_messages([
      ("system", template),
      ("human", query),
  ])

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 1.1 MB/s eta 0:00:00


In [3]:
import os

folder_name = 'pdf'
pdf_directory = '/content/' + folder_name

def import_files():
  files = []
  import_file()
  list = os.listdir(pdf_directory)
  for item in list:
    files.append(os.path.join(pdf_directory, item))
  return files
def import_file():
  from google.colab import files
  upload = files.upload(pdf_directory)

In [4]:
!pip install -q PyPDF2
import PyPDF2
from PyPDF2 import PdfReader
from langchain_core.documents import Document
import re

def get_documents_vector(files):
  documents = []
  text ='';
  for file in files:
    if file.endswith('.pdf'):
      with open(file, 'rb') as pdf_file:
          pdf_reader = PyPDF2.PdfReader(pdf_file)
          for page_num in range(len(pdf_reader.pages)):
                  page = pdf_reader.pages[page_num]
                  text += page.extract_text()
    text = re.sub(r'-?\d+(?:\.\d+)?', ' ', text)
    text = re.sub(r'\n', ' ', text)
    documents.append(Document(page_content=text))
  return documents

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.5 MB/s eta 0:00:00


In [5]:
from uuid import uuid4

def gen_documents_id(documents):
  return [str(uuid4()) for _ in range(len(documents))]

In [6]:
from langchain.schema import BaseOutputParser

class CommaSeparatedListOutputParser(BaseOutputParser):
  def parse(self, text: str):
    return text.strip().split(", ")

In [7]:
def do_query(results, query):
  conteudos = []
  for doc in results:
    conteudos.append(doc.page_content)

  template = "Utilize o contetúdo dos seguintes textos para responder a pergunta realizada: " + " ".join(conteudos)

  chat_prompt = create_chat_prompt(template, query)
  chat_model = create_chat_model()

  chain = chat_prompt | chat_model | CommaSeparatedListOutputParser()

  responses = chain.invoke({"text": query})

  print("\nResultado: \n")
  for response in responses:
    print("" + re.sub(r'-?\d+(?:\.\d+)?', ' ', response))

# Implamentations

In [ ]:
%env PINECONE_API_KEY=
pinecone_index_name = "tc3-as05"
pinecone_index = set_up_pinecone(pinecone_index_name)

In [ ]:
%env GOOGLE_API_KEY=
set_up_gemini()
embeddings = get_gemini_embeddings()

In [10]:
vector_store = get_vector_store(pinecone_index, embeddings)

In [11]:
while True:
  entrada = input("\n\nDigite no campo destacado\n- Digite 'sair' para encerrar\n- Digite 'anexar' para adicionar documentos\n- Ou digite que gostaria de saber sobre o conteúdo anexado\nEntrada: ")
  if entrada.lower() == 'sair':
    print("Encerrando o programa...")
    break
  elif entrada.lower() == 'anexar':
    files = import_files()
    documents_vector = get_documents_vector(files)
    documents_ids = gen_documents_id(documents_vector)
    vector_store.add_documents(documents=documents_vector, documents_ids=documents_ids)
  else:
    query = entrada
    results = vector_store.similarity_search(query=query,k=1)
    do_query(results, query)



Digite no campo destacado
- Digite 'sair' para encerrar
- Digite 'anexar' para adicionar documentos
- Ou digite que gostaria de saber sobre o conteúdo anexado
Entrada: anexar


Saving trabalho_de_conclusao_de_curso.pdf to /content/pdf/trabalho_de_conclusao_de_curso.pdf


Digite no campo destacado
- Digite 'sair' para encerrar
- Digite 'anexar' para adicionar documentos
- Ou digite que gostaria de saber sobre o conteúdo anexado
Entrada: Defina de forma resumida quais os benefícios do artigo sobre OCR.

Resultado: 

O artigo demonstra o desenvolvimento de um aplicativo móvel que usa OCR e TTS para auxiliar pessoas com deficiência visual e dificuldades de leitura.  O principal benefício é a ampliação do acesso à informação através da conversão de texto em imagens para áudio
promovendo maior autonomia e inclusão social.  Os testes mostraram a eficácia do sistema
especialmente quando comparado com outras bibliotecas OCR
destacando o potencial da solução como tecnologia assistiva.


Digite no campo destacado
- Digite 'sair' para encerrar
- Digite 'anexar' para adicionar documentos
- Ou digite que gostaria de saber sobre o conteúdo anexado
Entrada: sair
Encerrando o